import libraries and set style for matplotlib

In [9]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_rows', 200)

print(plt.style.available)
#plt.style.use('default')
#plt.style.use('seaborn-darkgrid')
#plt.style.use("seaborn")
plt.style.use("tableau-colorblind10")

['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


Read in data

In [10]:
df_all=pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp April\Python\web_scraping_project\data prepared for analysis\out_all.csv')
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138 entries, 0 to 2137
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              2138 non-null   int64  
 1   cat_name                2138 non-null   object 
 2   num_images              2138 non-null   int64  
 3   num_videos              2138 non-null   int64  
 4   age                     2138 non-null   object 
 5   breed                   2138 non-null   object 
 6   color                   1825 non-null   object 
 7   size                    2138 non-null   object 
 8   sex                     2138 non-null   object 
 9   characteristics         844 non-null    object 
 10  house_trained           2125 non-null   object 
 11  health                  2073 non-null   object 
 12  good_in_a_home_with     1337 non-null   object 
 13  adoption_fee            379 non-null    object 
 14  prefers_a_home_without  327 non-null    

In [11]:
df_all.head(2)

,Unnamed: 0,cat_name,num_images,num_videos,age,breed,color,size,sex,characteristics,house_trained,health,good_in_a_home_with,adoption_fee,prefers_a_home_without,coat_length,pet_story_num_words,rescue_group,data_set,num_cats
0,0,Ashe and Billie,6,0,Adult,Domestic Short Hair,NaN,Medium,Female,NaN,Yes,"Vaccinations up to date, spayed / neutered.",NaN,NaN,NaN,NaN,89.0,KittyKind,one_day,1
1,1,Billie and Ashe,6,0,Adult,Domestic Short Hair,NaN,Medium,Female,NaN,Yes,"Vaccinations up to date, spayed / neutered.",NaN,NaN,NaN,NaN,89.0,KittyKind,one_day,1


Create frequency distribution function

In [21]:
def proc_freq(df,variable_):  #Note variable_ must be given in quotes; example variable_: 'xyz'
    datax = df[variable_].value_counts().sort_index()
    
    datay = pd.DataFrame({
        variable_: datax.index,
        'Frequency': datax.values,
        'Percent': ((datax.values/datax.values.sum())*100).round(2),
        'Cumulative Frequency': datax.values.cumsum(),
        'Cumulative Percent': ((datax.values.cumsum()/datax.values.sum())*100).round(2)   })
    
    #datay.set_index(variable_)
    
    return(datay)

In [22]:
proc_freq(df_all,'data_set')

,data_set,Frequency,Percent,Cumulative Frequency,Cumulative Percent
0,fourteen_days,345,16.14,345,16.14
1,one_day,38,1.78,383,17.91
2,seven_days,204,9.54,587,27.46
3,spec_needs_other,14,0.65,601,28.11
4,thirty_day_spec_needs,104,4.86,705,32.97
5,thirty_days,1433,67.03,2138,100.00


In [23]:
df_all['cat_id']=df_all['cat_name'] + ' ' + df_all['rescue_group']

df_all['thirty_days']=df_all['data_set'].apply(lambda x:  x in ['thirty_days'])

df_all['other_days'] =df_all['data_set'].apply(lambda x: x in ['one_day','seven_days','fourteen_days'])      

df_all['spec_needs'] = df_all['data_set'].apply(lambda x: x in ['spec_needs_other','thirty_day_spec_needs'])

df_all['color'] = df_all['color'].astype(str)

Tabby = [True if 'Tabby' in cat else False for cat in df_all['color']]
print('length Tabby ',len(Tabby))
df_all['Tabby'] = pd.Series(Tabby)


df_all['pet_story_num_words']  = df_all['pet_story_num_words'].fillna(0)

df_all['without_story'] = df_all['pet_story_num_words'] == 0

df_all['coat_length_miss'] = df_all['coat_length'].fillna('unknown')

df_all['breed'] = df_all['breed'].fillna('unknown')

#create 'more_than_1_cat'
and1 = [True if 'and' in cat.split() else False for cat in df_all['cat_name']]
and2 = [True if '&' in cat.split() else False for cat in df_all['cat_name']]

mtn=(np.logical_or(and1,and2))
print('len more_than_1_cat ',len(mtn))

df_all['more_than_1_cat'] = pd.Series(mtn)



df_all['video']= df_all['num_videos'] > 0

df_all['num_pictures'] = df_all['num_images'].astype(str)

df_all['pictures'] = df_all['num_images'] > 0



breed_type=[]
for breed in df_all['breed']:
    if 'Domestic Short Hair' in breed:
        breed_type.append('DSH')
    elif 'Long Hair' in breed:   #Note 'Long Hair' must be identified before 'Mix','Tuxedo' 'Tortoiseshell' etc
        breed_type.append('DLH')
    elif 'Mix' in breed:
        breed_type.append('DSH')
    elif 'Tuxedo' in breed:
        breed_type.append('DSH')
    elif 'Tabby' in breed:
        breed_type.append('DSH')
    elif 'Calico' in breed:
        breed_type.append('DSH')
    elif 'Tortoiseshell' in breed:
        breed_type.append('DSH')
    elif 'Tiger' in breed:
         breed_type.append('DSH')
        
    else:
        breed_type.append('Other')

print('list length = ',len(breed_type))
        
df_all['breed_type'] = pd.Series(breed_type)

df_all.shape



length Tabby  2138
len more_than_1_cat  2138
list length =  2138


(2138, 32)

In [24]:
df_all2 = df_all.loc[df_all['spec_needs']==False,:]
df_all2 = df_all2.drop(['spec_needs','breed','size','Unnamed: 0','house_trained','characteristics','health','num_cats',], axis = 1)

df_all2.to_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp April\Python\web_scraping_project\data prepared for analysis\cleaned data\df_all.csv')

print('df_all2.shape = ',df_all2.shape)

#df_all2.info()

df_all2.shape =  (2020, 24)


In [17]:
#list_mask = list of rescue groups that have one or more duplicates
len(list_mask)
#final_mask 

189

In [25]:
#group_30b = df_thirty.groupby(['breed_type','age'])
mask = df_all2.groupby(['cat_id']).count()[['cat_name']].sort_values(by=['cat_name'],ascending=False) > 1

list_mask = df_all2.groupby(['cat_id']).count()[['cat_name']].sort_values(by=['cat_name'],ascending=False).loc[mask.values,:].index


final_mask = df_all2['cat_id'].isin(list_mask)


#the list created by below is the rescue group followed by however many duplicates
df_all2.loc[final_mask,['cat_id','rescue_group','thirty_days']].groupby(['thirty_days','rescue_group']).count()

cat_id
thirty_days rescue_group                                         
False       A Purrfect World                                    2
            Adopt-A-Buddy                                       2
            AdvoCat Rescue                                     48
            All About Cats Rescue, Inc                         23
            Alleykattz Rescue                                   4
            Animal Adoption Veterinary Center                   4
            Animal Life Savers Inc.                            20
            Animal Lighthouse Rescue                            2
            Animals Are Forever, Inc                            2
            Anjellicle Cats Rescue                              2
            Bergen County Animal Shelter                        3
            Best Friend Dog and Animal Adoption, Inc.           2
            Bronx Tails Cat Rescue                              6
            Brooklyn Adoptables                                 8
            Brooklyn Animal Action                             27
            Brooklyn Bridge Animal Welfare Coalition            8
            CLAWS                                               6
            Caring About The Strays (C.A.T.S.) Inc.             2
            Cat Assistance Inc.                                 2
            Cat Castle NYC                                      4
            City Critters Inc.                                  4
            East Coast Maine Coon Rescue                        4
            Eastchester Bay Stray Cat Project                   2
            Eccentric Kittens                                  16
            Edison Municipal Animal Shelter                    10
            For Blake's Sake Rescue, Inc.                       8
            Frankie's Fund for Feline Care and Rescue           2
            Friends of Linden Animal Shelter, Inc.             20
            Friends of the Clifton Animal Shelter               6
            Good World Animal Rescue and Protection,…           2
            Happy Friends Rescue                                2
            Happy Homes Animal Rescue                           8
            Hudson County Animal League (HCAL)                  2
            Humane Society of Westchester at New Rochelle       7
            Humane Urban Group                                  4
            Infinite Hope                                      10
            Jersey Tails                                        2
            JerseyCats                                          2
            Kat Connection                                      8
            KittyKaretakers of Queens                           2
            KittyKind                                           2
            Lend-a-Paw Inc.                                     4
            Liberty Humane Society                             16
            Meows Kitty Cat Rescue Inc.                        14
            Miss Pats Cats                                      2
            PAWS - Montclair                                    2
            Pequannock Animal Shelter                           2
            Pets4Luv Foundation                                 2
            Posh Pets Rescue                                    2
            Ready For Rescue                                    4
            Rescue Haven Foundation                             2
            S.A.R.A. Inc.                                       8
            Saving Cats & Kittens of Brooklyn                   2
            Sunset Park Cats                                    4
            The Lucky Cat Rescue                                3
            Union City Animal Trust                             3
            Union City Feral Cat Committee                     12
            Waggytail Rescue                                    2
            West Orange TNVR                                    2
            Whiskers-aGoGo Inc   

In [18]:
#df_all2.loc[df_all2['rescue_group']=='Animal Life Savers Inc.',['cat_name','data_set','age']]
df_all2.loc[df_all2['rescue_group']=='AdvoCat Rescue',['cat_name','data_set','age']]


,cat_name,data_set,age
51,Miss Misstoffelees,seven_days,Young
52,Fries & Mash - Bonded Brothers,seven_days,Kitten
53,Mash & Fries - Bonded Brothers,seven_days,Kitten
54,The Jimmy,seven_days,Kitten
55,Hermione - Hogwarts Kittens,seven_days,Kitten
56,Ron - Hogwarts Kittens,seven_days,Kitten
57,Harry - Hogwarts Kittens,seven_days,Kitten
108,Miss Misstoffelees,seven_days,Young
109,Fries & Mash - Bonded Brothers,seven_days,Kitten
110,Mash & Fries - Bonded Brothers,seven_days,Kitten


In [19]:
m_than_one = df_all.loc[df_all['cat_name'].str.contains('&| and ',case=False)]
m_than_one
df_all['more_than_1_cat'] = df_all['cat_name'].apply(lambda x:  x in m_than_one['cat_name'].to_list())
df_all['more_than_1_cat'].value_counts()


False    1921
True      217
Name: more_than_1_cat, dtype: int64

Create two separate DataFrames one for thirty days and one for less than thirty days. 

Create two subset DataFrames  to compare kittens only

In [20]:
df_thirty = df_all.loc[df_all.thirty_days,:]
df_thirty['days_on_site'] = 'Thirty Days Plus'

df_other_days = df_all.loc[df_all.other_days,:]
df_other_days['days_on_site'] = 'Less Than Thirty Days'

df_thirty_k = df_thirty.loc[df_thirty['age']=='Kitten',:]
df_other_days_k = df_other_days.loc[df_other_days['age']=='Kitten',:]

df_other_days['days_on_site'].value_counts()

combined_df = pd.concat([df_other_days,df_thirty], axis = 0, sort=False)

combined_df_k = combined_df.loc[combined_df['age']=='Kitten',:]

combined_df['days_on_site'].value_counts()

<ipython-input-20-46c2b61a3dfb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_thirty['days_on_site'] = 'Thirty Days Plus'


AttributeError: 'DataFrame' object has no attribute 'other_days'

In [ ]:
def func_freq(lst):
    for el in lst:
        print('Thirty Days Plus')
        print(proc_freq(df_thirty,el))
        print()
        print('Less Than Thirty Days')
        print(proc_freq(df_other_days,el))
        print('_'*75)

In [ ]:
cols=['age','sex','Tabby','without_story','coat_length_miss','coat_length', 'more_than_1_cat','breed_type','video', \
      'pictures','num_pictures']
func_freq(cols)

In [ ]:
def func_freq_k(lst):
    for el in lst:
        print('Thirty Days Plus Kittens')
        print(proc_freq(df_thirty_k,el))
        print()        print('Less Than Thirty Days Kittens')
        print(proc_freq(df_other_days_k,el))
        print('_'*75)

In [ ]:
cols=['age','sex','Tabby','without_story','coat_length_miss','coat_length', 'more_than_1_cat','breed_type','video', \
      'pictures','num_pictures']
func_freq_k(cols)

In [ ]:
print('df_other_days')
print(df_other_days.describe())
print('#'*20)
print('df_thirty')
print(df_thirty.describe())

Pet Story: Number Of Words Box Plot

In [ ]:
plt.style.use("tableau-colorblind10")

data1 = df_other_days['pet_story_num_words']
data2 = df_thirty['pet_story_num_words']

data = list([data1, data2])

# Figure size
plt.figure(figsize=(10,5))

fig, ax = plt.subplots()

# build a box plot
ax.boxplot(data)



# title and axis labels
ax.set_title('Pet Story: Number Of Words')
ax.set_xlabel('')
ax.set_ylabel('Number Of Words')
xticklabels=['Less Than Thirty Days', 'Thirty Days Plus']
ax.set_xticklabels(xticklabels)

plt.show()

In [ ]:
print('df_other_days median ',df_other_days['pet_story_num_words'].median())
print('df_other_days mean ',df_other_days['pet_story_num_words'].mean())
print(' ')
print('df_thirty median ',df_thirty['pet_story_num_words'].median())
print('df_thirty mean ',df_thirty['pet_story_num_words'].mean())


In [ ]:
df = combined_df.loc[:,['days_on_site','pet_story_num_words']]
data_wide = df.pivot(columns='days_on_site',
                     values='pet_story_num_words')
data_wide.head()

# calling density() to make
# multiple density plot 
data_wide.plot.density(figsize = (7, 7),
                       linewidth = 4)
  
plt.xlabel("Pet Story Number of Words")
data_wide

In [ ]:
#num_images

plt.style.use("tableau-colorblind10")

data1 = df_other_days['num_images']
data2 = df_thirty['num_images']

data = list([data1, data2])

# Figure size
plt.figure(figsize=(10,5))

fig, ax = plt.subplots()

# build a box plot
ax.boxplot(data)



# title and axis labels
ax.set_title('Number of Cat Pictures')
ax.set_xlabel('')
ax.set_ylabel('Pictures')
xticklabels=['Less Than Thirty Days', 'Thirty Days Plus']
ax.set_xticklabels(xticklabels)

plt.show()



In [ ]:
print('df_other_days median ',df_other_days['num_images'].median())
print('df_other_days mean ',df_other_days['num_images'].mean())
print(' ')
print('df_thirty median ',df_thirty['num_images'].median())
print('df_thirty mean ',df_thirty['num_images'].mean())


In [ ]:
df = combined_df.loc[:,['days_on_site','num_images']]
data_wide = df.pivot(columns='days_on_site',
                     values='num_images')
data_wide.head()

# calling density() to make
# multiple density plot 
data_wide.plot.density(figsize = (7, 7),
                       linewidth = 4)
  
plt.xlabel("Number of Pictures")

In [ ]:
g = sns.FacetGrid(penguins,col='island') #you can also use row
g.map_dataframe(sns.scatterplot, x='bill_depth_mm', y= 'bill_length_mm')
g.set_axis_labels('Culment Depth (mm)', 'Culmen Length (mm)')
g.set_titles(col_template= '{col_name} Island');

In [ ]:
print('30 days plus')
print(df_thirty['sex'].value_counts(normalize=True))
print('#'*50)
print('Other days')
print('_'*50)
print(df_other_days['sex'].value_counts(normalize=True)) 

In [ ]:
#plt.style.use('darkgrid')
#set_style('darkgrid[')]
plt.style.use("tableau-colorblind10")

In [ ]:
# Numbers of pairs of bars 
N = 2

# Data on X-axis

# Specify the values of blue bars (height)
Other_days = df_other_days['sex'].value_counts(normalize=True)*100
# Specify the values of orange bars (height)
Thirty_days = df_thirty['sex'].value_counts(normalize=True)*100

# Position of bars on x-axis
ind = np.arange(N)

# Figure size
plt.figure(figsize=(10,5))

# Width of a bar 
width = 0.3       

# Plotting
plt.bar(ind, Other_days , width )
plt.bar(ind + width, Thirty_days, width)

plt.xlabel('Gender')
plt.ylabel('Percentage Of Cats')
plt.title('Gender Comparison')

# xticks()
# First argument - A list of positions at which ticks should be placed
# Second argument -  A list of labels to place at the given locations
plt.xticks(ind + width / 2, ('Less Than Thirty Days','Thirty Days Plus'))

# Finding the best position for legends and putting it
plt.legend(loc=2)
plt.show()


In [ ]:
Other_days.

In [ ]:
Thirty_days

In [ ]:
df_thirty['breed_type'].value_counts(normalize=True)

In [ ]:
df_other_days['breed_type'].value_counts(normalize=True)

In [ ]:
group_30=df_thirty.groupby(['breed_type','age']).agg(['count'])['cat_name']
group_30.loc['DLH',:]

In [ ]:
dfb = df_thirty.groupby(['breed_type']).agg(['count'])['cat_name']
dfb.loc['DLH',:]

In [ ]:
group_30.loc['DLH',:]/dfb.loc['DLH',:]

In [ ]:
group_30b = df_thirty.groupby(['breed_type','age'])
group_30.agg(['sum'])['num_cats']

In [ ]:
df_thirty['age'].value_counts(normalize=True)

In [ ]:
df_other_days['age'].value_counts(normalize=True)

In [ ]:
df_other_days['age'].value_counts()

In [ ]:
#df_all.groupby('data_set')['age'].value_counts(normalize=True).plot.bar()

In [ ]:
#df_thirty['breed'].value_counts(normalize=True)


In [ ]:
df_thirty['color'].value_counts(normalize=True)

In [ ]:
df_other_days = df_all.loc[df_all.other_days,:]
df_other_days.head(2)
df_other_days['age'].value_counts(normalize=True)

In [ ]:
N=4

Thirty_Days_Plus = df_thirty['age'].value_counts(normalize=True)
Other_Days = df_other_days['age'].value_counts(normalize=True)

width=0.35
ind = np.arange(N) 


In [ ]:
N = 4

# Data on X-axis

# Specify the values of blue bars (height)
Other_Days = df_other_days['age'].value_counts(normalize=True)*100
# Specify the values of orange bars (height)
Thirty_Days_Plus = df_thirty['age'].value_counts(normalize=True)*100

# Position of bars on x-axis
ind = np.arange(N)

# Figure size
plt.figure(figsize=(10,5))

# Width of a bar 
width = 0.3       

# Plotting
plt.bar(ind, Other_Days , width, label='Less Than Thirty Days')
plt.bar(ind + width, Thirty_Days_Plus, width, label='Thirty Days Plus')

plt.xlabel('Age')
plt.ylabel('Percentage Of Cats')
plt.title('Age Comparison')

# xticks()
# First argument - A list of positions at which ticks should be placed
# Second argument -  A list of labels to place at the given locations
plt.xticks(ind + width / 2, ('Kitten', 'Young', 'Adult', 'Senior'))

# Finding the best position for legends and putting it
plt.legend(loc='best')
plt.show()

In [ ]:
df_other_days['color'].value_counts(normalize=True)

In [ ]:
rescue_groups_other = list(df_other_days['rescue_group'].unique())
#type(df_other_days['rescue_group'].unique())
df_other_rescue = pd.DataFrame({'rescue_group':rescue_groups_other})
df_other_rescue

In [ ]:
df_all.groupby(['rescue_group'])['unique_cat_id'].nunique()

In [ ]:
df_days = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp April\Python\web_scraping_project\data prepared for analysis\out_days_petfinder_combined.csv')

df_days.head(2) 

In [ ]:

df_spec_needs = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp April\Python\web_scraping_project\data prepared for analysis\out_spec_needs_combined.csv')
df_spec_needs.head(2)

In [ ]:
df_spec_needs = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp April\Python\web_scraping_project\data prepared for analysis\out_spec_needs_combined.csv')